## Ingest required fake data into MYSQL database

In [ ]:
from pathlib import Path
import sys

cwd = Path.cwd().parent.joinpath("money_map")
sys.path.append(cwd)

from money_map.ingestion.rbpn_ingestion import RBPN_Ingestor
from money_map.ingestion.categories_ingestion import Categories_Ingestor
from money_map.pipelines.transactions import recreate_transactions_labeled

-  Ingest fake RBPN transaction data:

In [ ]:
# define target path
file_name = "fake_rbpn_data.csv"
rbpn_file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)

# init ingestor
ingestor = RBPN_Ingestor()
ingestor.connector.create_tables()

#ingest data
ingestor.ingest_data(file_path=rbpn_file_path)

-  Ingest categories for labeling transactions:

In [ ]:
file_name = "fake_categories.json"
file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)

# init
cat_ingestor = Categories_Ingestor()
cat_ingestor.connector.create_tables()

#ingest data
cat_ingestor.ingest_data(file_path=file_path)


-  Ingest fake labeled data

In [ ]:
import pandas as pd
file_name = "fake_rbpn_participants_labeled.csv"
file_path = Path.cwd().parent.joinpath("data/interim").joinpath(file_name)


df = pd.read_csv(file_path,sep=";")
df.head(5)

In [ ]:
df.to_sql(con=ingestor.connector.create_sql_engine(),
            name="participants_labeled",
            if_exists="replace",
            index=False)

In [ ]:
recreate_transactions_labeled(engine=ingestor.connector.create_sql_engine())